In [41]:
using RxInfer, Random, LinearAlgebra, Plots, Flux, Plots, StableRNGs, SpecialFunctions, Rocket
using CCMP
using Turing
using DynamicHMC, AdvancedVI
using BenchmarkTools
pgfplotsx()

Plots.PGFPlotsXBackend()

In [42]:
# data generating process
nr_observations = 50
reference_point = 53
hidden_location = collect(1:nr_observations) + rand(MersenneTwister(124), NormalMeanVariance(0.0, sqrt(100)), nr_observations)
measurements = (hidden_location .- reference_point).^2 + rand(MersenneTwister(124), NormalMeanVariance(0.0, 200), nr_observations);

In [43]:
# plot hidden location and reference frame
p1 = plot(1:nr_observations, hidden_location, linewidth=3, legend=:topleft, label="hidden location")
hline!([reference_point], linewidth=3, label="reference point")
xlabel!("time [sec]"), ylabel!("location [cm]")

# plot measurements
p2 = scatter(1:nr_observations, measurements, linewidth=3, label="measurements")
xlabel!("time [sec]"), ylabel!("squared distance [cm2]")

p = plot(p1, p2)
savefig(p,"square_observations.tex")

"/Users/sabinamakhmudova/repos/CVMP/demos/square_observations.tex"

In [38]:
function compute_squared_distance(z)
    (z - reference_point)^2
end;

In [39]:
RxInfer.@model function measurement_model(nr_observations)

    # allocate random variables and observations
    z = randomvar(nr_observations)
    y = datavar(Float64, nr_observations)

    # set priors on precision parameters
    τ ~ Gamma(shape = 1.0, rate = 1.0e-12)
    γ ~ Gamma(shape = 1.0, rate = 1.0e-12)
    
    # specify estimate of initial location
    z[1] ~ Normal(mean = 0, precision = τ)
    y[1] ~ Normal(mean = compute_squared_distance(z[1]), precision = γ)

    # loop over observations
    for t in 2:nr_observations

        # specify state transition model
        z[t] ~ Normal(mean = z[t-1] + 1, precision = τ)

        # specify non-linear observation model
        y[t] ~ Normal(mean = compute_squared_distance(z[t]), precision = γ)
        
    end

end

In [30]:
@meta function measurement_meta(rng, nr_samples, nr_iterations, optimizer)
    compute_squared_distance() -> CVI(rng, nr_samples, nr_iterations, optimizer, ForwardDiffGrad(), 10, Val(false), false)
end;

In [31]:
@constraints function measurement_constraints()
    q(z, τ, γ) = q(z)q(τ)q(γ)
end;

In [32]:
results_cvi = inference(
    model = measurement_model(nr_observations),
    data = (y = measurements,),
    iterations = 50,
    free_energy = true,
    returnvars = (z = KeepLast(),),
    constraints = measurement_constraints(),
    meta = measurement_meta(StableRNG(42), 200, 10, Descent(0.001)),
    initmessages = (z = NormalMeanVariance(0, 5),),
    initmarginals = (z = NormalMeanVariance(0, 5), τ = GammaShapeRate(1.0, 1000), γ = GammaShapeRate(1.0, 1000),),
)

Inference results:
  Posteriors       | available for (z)
  Free Energy:     | Real[522.916, 439.434, 423.231, 421.204, 419.006, 416.184, 414.074, 412.109, 410.061, 410.146  …  392.381, 392.179, 391.79, 391.988, 392.112, 392.069, 391.904, 391.801, 391.712, 391.506]


In [13]:
@benchmark inference(
    model = measurement_model(nr_observations),
    data = (y = measurements,),
    iterations = 50,
    free_energy = true,
    returnvars = (z = KeepLast(),),
    constraints = measurement_constraints(),
    meta = measurement_meta(StableRNG(42), 200, 10, Descent(0.001)),
    initmessages = (z = NormalMeanVariance(0, 5),),
    initmarginals = (z = NormalMeanVariance(0, 5), τ = GammaShapeRate(1.0, 1000), γ = GammaShapeRate(1.0, 1000),),
)

BenchmarkTools.Trial: 19 samples with 1 evaluation.
 Range (min … max):  257.721 ms … 317.608 ms  ┊ GC (min … max): 6.38% … 8.07%
 Time  (median):     262.920 ms               ┊ GC (median):    6.41%
 Time  (mean ± σ):   268.730 ms ±  14.205 ms  ┊ GC (mean ± σ):  8.27% ± 2.56%

  ██               ▁ ▄                                           
  ██▁▁▆▆▁▆▁▁▁▁▁▁▁▆▁█▆█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  258 ms           Histogram: frequency by time          318 ms <

 Memory estimate: 257.80 MiB, allocs estimate: 9209792.

In [14]:
@rule DeltaFn(:out, Marginalisation) (q_ins::ReactiveMP.FactorizedJoint{P}, meta::ReactiveMP.DeltaMeta{M}) where {P <: NTuple{1}, M <: CVI} = begin
    method = ReactiveMP.getmethod(meta)
    node = getnode()
    m_out = Rocket.getrecent(ReactiveMP.messagein(node.out))
    m_ins = Rocket.getrecent.(ReactiveMP.messagein.(node.ins))
    q_y_params = naturalparams(prod(method, getdata(m_ins[1]), getdata(m_out), q_ins[1], getnodefn(Val(:out))))
    return convert(Distribution, q_y_params - naturalparams(getdata(m_out)))
end

In [15]:
results_ccmp = inference(
    model = measurement_model(nr_observations),
    data = (y = measurements,),
    iterations = 50,
    free_energy = true,
    returnvars = (z = KeepLast(),),
    constraints = measurement_constraints(),
    meta = measurement_meta(StableRNG(42), 200, 10, Descent(0.008)),
    initmessages = (z = NormalMeanVariance(0, 5),),
    initmarginals = (z = NormalMeanVariance(0, 5), τ = GammaShapeRate(1.0, 1000), γ = GammaShapeRate(1.0, 1000),),
)

Inference results:
  Posteriors       | available for (z)
  Free Energy:     | Real[559.26, 438.928, 423.214, 418.7, 414.281, 409.274, 405.503, 403.736, 401.308, 401.338  …  391.93, 391.801, 391.658, 391.622, 391.53, 391.444, 391.322, 391.218, 391.21, 391.044]


In [16]:
@meta function linear_measurement_meta()
    compute_squared_distance() -> Linearization()
end;

In [33]:
results_linear = inference(
    model = measurement_model(nr_observations),
    data = (y = measurements,),
    iterations = 50,
    free_energy = true,
    returnvars = (z = KeepLast(),),
    constraints = measurement_constraints(),
    meta = linear_measurement_meta(),
    initmessages = (z = NormalMeanVariance(0, 5),),
    initmarginals = (z = NormalMeanVariance(0, 5), τ = GammaShapeRate(1, 1000), γ = GammaShapeRate(1, 1000),),
)

Inference results:
  Posteriors       | available for (z)
  Free Energy:     | Real[421.528, 411.335, 401.848, 394.865, 392.951, 392.594, 392.429, 392.321, 392.237, 392.16  …  390.553, 390.51, 390.469, 390.427, 390.385, 390.344, 390.303, 390.262, 390.221, 390.18]


In [52]:
@benchmark inference(
    model = measurement_model(nr_observations),
    data = (y = measurements,),
    iterations = 50,
    free_energy = true,
    returnvars = (z = KeepLast(),),
    constraints = measurement_constraints(),
    meta = linear_measurement_meta(),
    initmessages = (z = NormalMeanVariance(0, 100),),
    initmarginals = (z = NormalMeanVariance(0, 100), τ = GammaShapeRate(1, 1000), γ = GammaShapeRate(1, 1000),),
)

BenchmarkTools.Trial: 95 samples with 1 evaluation.
 Range (min … max):  49.456 ms … 74.071 ms  ┊ GC (min … max): 0.00% … 29.71%
 Time  (median):     50.682 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.696 ms ±  6.684 ms  ┊ GC (mean ± σ):  3.87% ±  8.66%

  ▂▁▅█                                                         
  ████▇▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▁▃▄ ▁
  49.5 ms         Histogram: frequency by time          74 ms <

 Memory estimate: 21.48 MiB, allocs estimate: 637738.

In [18]:
# Turing model definition.
Turing.@model function NonlinearFiltering(y, r_point)
    # Get observation length.
    N = length(y)

    # State sequence.
    s = tzeros(Float64, N)

    # set priors on precision parameters
    τ ~ Gamma(1.0, 1.0e-12)
    γ ~ Gamma(1.0, 1.0e-12)

    # Observe each point of the input.
    s[1] ~ Turing.Normal(0, τ)
    y[1] ~ Turing.Normal((s[1]-r_point)^2, γ)

    for i in 2:N
        s[i] ~ Turing.Normal(s[i-1] + 1, τ)
        y[i] ~ Turing.Normal((s[i]-r_point)^2, γ)
    end
end;
model = NonlinearFiltering(measurements, reference_point)

DynamicPPL.Model{typeof(NonlinearFiltering), (:y, :r_point), (), (), Tuple{Vector{Float64}, Int64}, Tuple{}, DynamicPPL.DefaultContext}(NonlinearFiltering, (y = [2470.235393733442, 2325.219490861669, 2390.1960804010614, 2493.140286092835, 2430.9281458279984, 2167.0752578998745, 2635.7796600262977, 2014.537261227006, 1723.287794104091, 2081.343956668728  …  163.5366273198292, 220.823980051994, 51.75194422139158, 38.446795064708645, 31.410193321197557, 28.7123839864023, 83.12669430851457, 20.559231292389978, 13.739213968976847, 14.750756708287287], r_point = 53), NamedTuple(), DynamicPPL.DefaultContext())

In [19]:
advi = ADVI(100, 100)
q = vi(model, advi);
AdvancedVI.elbo(advi, q, model, 100000)

┌ Info: [ADVI] Should only be seen once: optimizer created for θ
│   objectid(θ) = 13362888352744503043
└ @ AdvancedVI /Users/sabinamakhmudova/.julia/packages/AdvancedVI/hVQ2g/src/AdvancedVI.jl:202


[ADVI] Optimizing...   2%  ETA: 0:01:19

[ADVI] Optimizing...  23%  ETA: 0:00:11

[ADVI] Optimizing...  40%  ETA: 0:00:07

[ADVI] Optimizing...  55%  ETA: 0:00:04

[ADVI] Optimizing...  70%  ETA: 0:00:03

[ADVI] Optimizing...  85%  ETA: 0:00:01

[ADVI] Optimizing... 100% Time: 0:00:08


-1.6532924652713926e12

In [49]:
@benchmark vi(model, advi)

┌ Info: [ADVI] Should only be seen once: optimizer created for θ
│   objectid(θ) = 4158690067138828297
└ @ AdvancedVI /Users/mykola/.julia/packages/AdvancedVI/hVQ2g/src/AdvancedVI.jl:202


[ADVI] Optimizing...  13%  ETA: 0:00:07

[ADVI] Optimizing...  27%  ETA: 0:00:06

[ADVI] Optimizing...  40%  ETA: 0:00:05

[ADVI] Optimizing...  53%  ETA: 0:00:04

...

[ADVI] Optimizing...  58%  ETA: 0:00:03

[ADVI] Optimizing...  72%  ETA: 0:00:02

[ADVI] Optimizing...  86%  ETA: 0:00:01

[ADVI] Optimizing... 100% Time: 0:00:07


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 7.219 s (27.34% GC) to evaluate,
 with a memory estimate of 25.29 GiB, over 71772585 allocations.

In [20]:
chn = sample(model, DynamicNUTS(), 3000);
chn2 = sample(model, HMC(0.01, 1000), 1000);

Sampling   0%|                                          |  ETA: N/A


Sampling   0%|▎                                         |  ETA: 0:12:56
Sampling   1%|▍                                         |  ETA: 0:06:27
Sampling   2%|▋                                         |  ETA: 0:04:17
Sampling   2%|▉                                         |  ETA: 0:03:12
Sampling   2%|█                                         |  ETA: 0:02:33
Sampling   3%|█▎                                        |  ETA: 0:02:07
Sampling   4%|█▌                                        |  ETA: 0:01:48
Sampling   4%|█▋                                        |  ETA: 0:01:34
Sampling   4%|█▉                                        |  ETA: 0:01:23
Sampling   5%|██▏                                       |  ETA: 0:01:15
Sampling   6%|██▎                                       |  ETA: 0:01:07
Sampling   6%|██▌                                       |  ETA: 0:01:02
Sampling   6%|██▊                                       |  ETA: 0:00:57
Sampling   7%|███                                       |  ETA: 

Sampling  18%|███████▊                                  |  ETA: 0:00:18
Sampling  19%|████████                                  |  ETA: 0:00:17
Sampling  20%|████████▎                                 |  ETA: 0:00:17
Sampling  20%|████████▍                                 |  ETA: 0:00:16
Sampling  20%|████████▋                                 |  ETA: 0:00:16
Sampling  21%|████████▉                                 |  ETA: 0:00:15
Sampling  22%|█████████                                 |  ETA: 0:00:15
Sampling  22%|█████████▎                                |  ETA: 0:00:14
Sampling  22%|█████████▌                                |  ETA: 0:00:14
Sampling  23%|█████████▋                                |  ETA: 0:00:14
Sampling  24%|█████████▉                                |  ETA: 0:00:13
Sampling  24%|██████████▏                               |  ETA: 0:00:13
Sampling  24%|██████████▎                               |  ETA: 0:00:12
Sampling  25%|██████████▌                               |  ETA: 

Sampling  40%|█████████████████                         |  ETA: 0:00:06
Sampling  41%|█████████████████▎                        |  ETA: 0:00:06
Sampling  42%|█████████████████▍                        |  ETA: 0:00:06
Sampling  42%|█████████████████▋                        |  ETA: 0:00:06
Sampling  42%|█████████████████▉                        |  ETA: 0:00:06
Sampling  43%|██████████████████                        |  ETA: 0:00:05
Sampling  44%|██████████████████▎                       |  ETA: 0:00:05
Sampling  44%|██████████████████▌                       |  ETA: 0:00:05
Sampling  44%|██████████████████▊                       |  ETA: 0:00:05
Sampling  45%|██████████████████▉                       |  ETA: 0:00:05
Sampling  46%|███████████████████▏                      |  ETA: 0:00:05
Sampling  46%|███████████████████▍                      |  ETA: 0:00:05
Sampling  46%|███████████████████▌                      |  ETA: 0:00:05
Sampling  47%|███████████████████▊                      |  ETA: 

Sampling  56%|███████████████████████▌                  |  ETA: 0:00:03
Sampling  56%|███████████████████████▊                  |  ETA: 0:00:03
Sampling  57%|████████████████████████                  |  ETA: 0:00:03
Sampling  57%|████████████████████████▏                 |  ETA: 0:00:03
Sampling  58%|████████████████████████▍                 |  ETA: 0:00:03
Sampling  58%|████████████████████████▋                 |  ETA: 0:00:03
Sampling  59%|████████████████████████▊                 |  ETA: 0:00:03
Sampling  60%|█████████████████████████                 |  ETA: 0:00:03
Sampling  60%|█████████████████████████▎                |  ETA: 0:00:03
Sampling  60%|█████████████████████████▍                |  ETA: 0:00:03
Sampling  61%|█████████████████████████▋                |  ETA: 0:00:03
Sampling  62%|█████████████████████████▉                |  ETA: 0:00:03
Sampling  62%|██████████████████████████                |  ETA: 0:00:03
Sampling  62%|██████████████████████████▎               |  ETA: 

...

Sampling   0%|                                          |  ETA: N/A


Sampling   0%|▎                                         |  ETA: 0:06:14
Sampling   1%|▍                                         |  ETA: 0:03:06
Sampling   2%|▋                                         |  ETA: 0:02:03
Sampling   2%|▉                                         |  ETA: 0:01:32
Sampling   2%|█                                         |  ETA: 0:01:13
Sampling   3%|█▎                                        |  ETA: 0:01:01
Sampling   4%|█▌                                        |  ETA: 0:00:52
Sampling   4%|█▋                                        |  ETA: 0:00:45
Sampling   4%|█▉                                        |  ETA: 0:00:40
Sampling   5%|██▏                                       |  ETA: 0:00:36
Sampling   6%|██▎                                       |  ETA: 0:00:32
Sampling   6%|██▌                                       |  ETA: 0:00:30
Sampling   6%|██▊                                       |  ETA: 0:00:27
Sampling   7%|███                                       |  ETA: 

Sampling  28%|███████████▌                              |  ETA: 0:00:05
Sampling  28%|███████████▊                              |  ETA: 0:00:05
Sampling  28%|████████████                              |  ETA: 0:00:05
Sampling  29%|████████████▏                             |  ETA: 0:00:05
Sampling  30%|████████████▍                             |  ETA: 0:00:05
Sampling  30%|████████████▋                             |  ETA: 0:00:05
Sampling  30%|████████████▊                             |  ETA: 0:00:05
Sampling  31%|█████████████                             |  ETA: 0:00:05
Sampling  32%|█████████████▎                            |  ETA: 0:00:04
Sampling  32%|█████████████▌                            |  ETA: 0:00:04
Sampling  32%|█████████████▋                            |  ETA: 0:00:04
Sampling  33%|█████████████▉                            |  ETA: 0:00:04
Sampling  34%|██████████████▏                           |  ETA: 0:00:04
Sampling  34%|██████████████▎                           |  ETA: 

Sampling  70%|█████████████████████████████▎            |  ETA: 0:00:01
Sampling  70%|█████████████████████████████▍            |  ETA: 0:00:01
Sampling  70%|█████████████████████████████▋            |  ETA: 0:00:01
Sampling  71%|█████████████████████████████▉            |  ETA: 0:00:01
Sampling  72%|██████████████████████████████            |  ETA: 0:00:01
Sampling  72%|██████████████████████████████▎           |  ETA: 0:00:01
Sampling  72%|██████████████████████████████▌           |  ETA: 0:00:01
Sampling  73%|██████████████████████████████▋           |  ETA: 0:00:01
Sampling  74%|██████████████████████████████▉           |  ETA: 0:00:01
Sampling  74%|███████████████████████████████▏          |  ETA: 0:00:01
Sampling  74%|███████████████████████████████▎          |  ETA: 0:00:01
Sampling  75%|███████████████████████████████▌          |  ETA: 0:00:01
Sampling  76%|███████████████████████████████▊          |  ETA: 0:00:01
Sampling  76%|███████████████████████████████▉          |  ETA: 

Sampling  80%|█████████████████████████████████▍        |  ETA: 0:00:01
Sampling  80%|█████████████████████████████████▋        |  ETA: 0:00:01
Sampling  80%|█████████████████████████████████▊        |  ETA: 0:00:01
Sampling  81%|██████████████████████████████████        |  ETA: 0:00:01
Sampling  82%|██████████████████████████████████▎       |  ETA: 0:00:00
Sampling  82%|██████████████████████████████████▌       |  ETA: 0:00:00
Sampling  82%|██████████████████████████████████▋       |  ETA: 0:00:00
Sampling  83%|██████████████████████████████████▉       |  ETA: 0:00:00
Sampling  84%|███████████████████████████████████▏      |  ETA: 0:00:00
Sampling  84%|███████████████████████████████████▎      |  ETA: 0:00:00
Sampling  84%|███████████████████████████████████▌      |  ETA: 0:00:00
Sampling  85%|███████████████████████████████████▊      |  ETA: 0:00:00
Sampling  86%|███████████████████████████████████▉      |  ETA: 0:00:00
Sampling  86%|████████████████████████████████████▏     |  ETA: 

In [54]:
@benchmark sample(model, DynamicNUTS(), 3000)

Sampling   0%|                                          |  ETA: N/A


Sampling   0%|▎                                         |  ETA: 0:02:08
Sampling   1%|▍                                         |  ETA: 0:01:04
Sampling   2%|▋                                         |  ETA: 0:00:43
Sampling   2%|▉                                         |  ETA: 0:00:32
Sampling   2%|█                                         |  ETA: 0:00:26
Sampling   3%|█▎                                        |  ETA: 0:00:21
Sampling   4%|█▌                                        |  ETA: 0:00:18
Sampling   4%|█▋                                        |  ETA: 0:00:16
Sampling   4%|█▉                                        |  ETA: 0:00:14
Sampling   5%|██▏                                       |  ETA: 0:00:13
Sampling   6%|██▎                                       |  ETA: 0:00:12
Sampling   6%|██▌                                       |  ETA: 0:00:11
Sampling   6%|██▊                                       |  ETA: 0:00:10
Sampling   7%|███                                       |  ETA: 

Sampling  14%|█████▉                                    |  ETA: 0:00:05
Sampling  14%|██████▏                                   |  ETA: 0:00:04
Sampling  15%|██████▎                                   |  ETA: 0:00:04
Sampling  16%|██████▌                                   |  ETA: 0:00:04
Sampling  16%|██████▊                                   |  ETA: 0:00:04
Sampling  16%|██████▉                                   |  ETA: 0:00:04
Sampling  17%|███████▏                                  |  ETA: 0:00:04
Sampling  18%|███████▍                                  |  ETA: 0:00:04
Sampling  18%|███████▌                                  |  ETA: 0:00:03
Sampling  18%|███████▊                                  |  ETA: 0:00:03
Sampling  19%|████████                                  |  ETA: 0:00:03
Sampling  20%|████████▎                                 |  ETA: 0:00:03
Sampling  20%|████████▍                                 |  ETA: 0:00:03
Sampling  20%|████████▋                                 |  ETA: 

Sampling  31%|█████████████                             |  ETA: 0:00:02
Sampling  32%|█████████████▎                            |  ETA: 0:00:02
Sampling  32%|█████████████▌                            |  ETA: 0:00:02
Sampling  32%|█████████████▋                            |  ETA: 0:00:02
Sampling  33%|█████████████▉                            |  ETA: 0:00:02
Sampling  34%|██████████████▏                           |  ETA: 0:00:02
Sampling  34%|██████████████▎                           |  ETA: 0:00:02
Sampling  34%|██████████████▌                           |  ETA: 0:00:02
Sampling  35%|██████████████▊                           |  ETA: 0:00:02
Sampling  36%|██████████████▉                           |  ETA: 0:00:02
Sampling  36%|███████████████▏                          |  ETA: 0:00:02
Sampling  36%|███████████████▍                          |  ETA: 0:00:02
Sampling  37%|███████████████▌                          |  ETA: 0:00:01
Sampling  38%|███████████████▊                          |  ETA: 

Sampling  44%|██████████████████▌                       |  ETA: 0:00:01
Sampling  44%|██████████████████▊                       |  ETA: 0:00:01
Sampling  45%|██████████████████▉                       |  ETA: 0:00:01
Sampling  46%|███████████████████▏                      |  ETA: 0:00:01
Sampling  46%|███████████████████▍                      |  ETA: 0:00:01
Sampling  46%|███████████████████▌                      |  ETA: 0:00:01
Sampling  47%|███████████████████▊                      |  ETA: 0:00:01
Sampling  48%|████████████████████                      |  ETA: 0:00:01
Sampling  48%|████████████████████▏                     |  ETA: 0:00:01
Sampling  48%|████████████████████▍                     |  ETA: 0:00:01
Sampling  49%|████████████████████▋                     |  ETA: 0:00:01
Sampling  50%|████████████████████▊                     |  ETA: 0:00:01
Sampling  50%|█████████████████████                     |  ETA: 0:00:01
Sampling  50%|█████████████████████▎                    |  ETA: 

Excessive output truncated after 524383 bytes.

Sampling  61%|█████████████████████████▋                |  ETA: 0:00:01
Sampling  62%|█████████████████████████▉                |  ETA: 0:00:01
Sampling  62%|██████████████████████████                |  ETA: 0:00:01
Sampling  62%|██████████████████████████▎               |  ETA: 0:00:01
Sampling  63%|██████████████████████████▌               |  ETA: 0:00:01
Sampling  64%|██████████████████████████▋               |  ETA: 0:00:01
Sampling  64%|██████████████████████████▉               |  ETA: 0:00:01
Sampling  64%|███████████████████████████▏              |  ETA: 0:00:01
Sampling  65%|███████████████████████████▎              |  ETA: 0:00:01
Sampling  66%|███████████████████████████▌              |  ETA: 0:00:01
Sampling  66%|███████████████████████████▊              |  ETA: 0:00:01
Sampling  66%|███████████████████████████▉              |  ETA: 0:00:01
Sampling  67%|████████████████████████████▏             |  ETA: 0:00:01
Sampling  68%|████████████████████████████▍             |  ETA: 

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.371 s …    2.262 s  ┊ GC (min … max):  8.76% … 9.80%
 Time  (median):     1.501 s               ┊ GC (median):    10.72%
 Time  (mean ± σ):   1.711 s ± 481.436 ms  ┊ GC (mean ± σ):   9.79% ± 0.98%

  █       █                                                █  
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.37 s         Histogram: frequency by time         2.26 s <

 Memory estimate: 1.61 GiB, allocs estimate: 9404927.

In [58]:
@benchmark sample(model, HMC(0.01, 1000), 3000)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  829.076 ms … 863.968 ms  ┊ GC (min … max):  8.66% … 12.37%
 Time  (median):     841.983 ms               ┊ GC (median):    10.56%
 Time  (mean ± σ):   844.056 ms ±  12.544 ms  ┊ GC (mean ± σ):  10.44% ±  1.25%

  █         █       █       █             █                   █  
  █▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  829 ms           Histogram: frequency by time          864 ms <

 Memory estimate: 934.72 MiB, allocs estimate: 6611421.

In [21]:
q_samples = rand(q, 1000)
s_posteriors = Vector{Any}(undef,50)
s2_posteriors = Vector{Any}(undef,50)
q_s_posteriors = Vector{Any}(undef,50)
for i in 1:50
    s_posteriors[i] = chn["s[$(i)]"]
    s2_posteriors[i] = chn2["s[$(i)]"]
    q_s_posteriors[i] = q_samples[i,1:1000]
end

In [44]:
# plot estimates for location
p1 = plot(collect(1:nr_observations), hidden_location, label = "hidden location", legend=:topleft, linewidth=3, color = :red)
plot!(map(mean, results_cvi.posteriors[:z]), label = "estimated location CVI (±2σ)", ribbon = map(x -> 2*std(x), results_cvi.posteriors[:z]), fillalpha=0.5, linewidth=3, color = :orange)
plot!(map(mean, results_ccmp.posteriors[:z]), label = "estimated location CVMP (±2σ)", ribbon = map(x -> 2*std(x), results_ccmp.posteriors[:z]), fillalpha=0.5, linewidth=3, color = :green)
plot!(map(mean, results_linear.posteriors[:z]), label = "estimated location Linear (±2σ)", ribbon = map(x -> 2*std(x), results_linear.posteriors[:z]), fillalpha=0.5, linewidth=3, color = :blue)
plot!(map(mean, q_s_posteriors), label = "estimated location Turing ADVI (±2σ)", ribbon = map(x -> 2*std(x), q_s_posteriors), fillalpha=0.5, linewidth=3, color = :purple)
plot!(map(mean, s_posteriors), label = "estimated location Turing NUTS (±2σ)", ribbon = map(x -> 2*std(x), s_posteriors), fillalpha=0.5, linewidth=3, color = :pink)
plot!(map(mean, s2_posteriors), label = "estimated location Turing HMC (±2σ)", ribbon = map(x -> 2*std(x), s2_posteriors), fillalpha=0.5, linewidth=3, color = :yellow)
xlabel!("time [sec]"), ylabel!("location [cm]")

# # plot Bethe free energy
# p2 = plot(results_cvi.free_energy, linewidth=3, label = "CVI")
# p2 = plot(results_ccmp.free_energy, linewidth=3, label = "CVMP")
# p2 = plot!(results_linear.free_energy, linewidth=3, label = "Linear")
# xlabel!("iteration"), ylabel!("Bethe free energy [nats]")

plot(p1)
savefig(p1, "square_results.tex")

"/Users/sabinamakhmudova/repos/CVMP/demos/square_results.tex"

In [63]:
mean(map(x -> 2*std(x), results_linear.posteriors[:z]))

0.7442179352660954

In [61]:
mean(map(x -> 2*std(x), results_cvi.posteriors[:z]))

7.102987832018541

In [62]:
mean(map(x -> 2*std(x), results_ccmp.posteriors[:z]))

3.112909171377821

In [64]:
mean(map(x -> 2*std(x), q_s_posteriors))

0.7241894114453199

In [65]:
mean(map(x -> 2*std(x), s_posteriors))

5.092609185627203e-15

In [66]:
mean(map(x -> 2*std(x), s2_posteriors))

6.7229871909330755e-15

In [46]:
savefig("square_observations_results")

"/Users/mykola/repos/CIExpirements/demos/square_observations_results.png"